#### Data Pre-process

In [16]:
import pandas as pd
import os

#Load dataframe dari file csv
dataframe1 = pd.read_csv('Csv-data/tempat-penampungan-sampah-sementara-tps-di-wilayah-dki-jakarta-tahun-2019.csv')
data1 = dataframe1.loc[:,['wilayah','kecamatan']]
data2 = dataframe1.loc[:,['kecamatan','volume_sampah_rata_rata_per_hari(m3)']]
data2.to_csv('Csv-data/volume.csv',index=False)

In [5]:
#Pre-process data before filtering
for column_name, column_data in data1.items():
    data1[column_name] = data1[column_name].str.strip()
    data1[column_name] = data1[column_name].str.title()

reduced_data = data1.drop_duplicates()

In [7]:
#Sort data based on alphabetical order
sorted_data = reduced_data.sort_values(by=['wilayah','kecamatan'])
sorted_data.to_csv('Csv-data/sorted.csv', index=False)

#there are some typo that should be handled manually

In [10]:
#count data for amaount of tps
data_duplicate = data1['kecamatan'].value_counts()
data_count = pd.DataFrame({'kecamatan':data_duplicate.index, 'Jumlah':data_duplicate.values})

counted_data = data_count.sort_values(by=['kecamatan'])
counted_data.to_csv('Csv-data/counted.csv',index=False)

#there are some typo that should be handled manually

In [ ]:
#merge data from counting and sorting
final_sorted = pd.read_csv('Csv-data/sorted-final.csv')
final_counted = pd.read_csv('Csv-data/counted-final.csv')

merged = pd.merge(final_sorted,final_counted, on='kecamatan')

merged.to_csv('Csv-data/merged1.csv',index=False)

In [18]:
#before load data, correct some typo from the original csv file
volume = pd.read_csv('Csv-data/volume-final.csv')

#pre-process data before using it
volume['kecamatan'] = volume['kecamatan'].str.strip()
volume['kecamatan'] = volume['kecamatan'].str.title()
volume['volume_sampah_rata_rata_per_hari(m3)'] = volume['volume_sampah_rata_rata_per_hari(m3)'].astype(float)

#data processing
grouped_sum = volume.groupby('kecamatan')['volume_sampah_rata_rata_per_hari(m3)'].transform('sum')
volume['total_volume(m3)'] = grouped_sum

In [19]:
#select only necessary and delete duplicate data
total_volume = volume.loc[:,['kecamatan','total_volume(m3)']]
reduced_volume = total_volume.drop_duplicates()

reduced_volume.to_csv('Csv-data/reduced-volume.csv',index=False)
#there are some typo that should be handled manually

In [24]:
#Load dataframe from csv file
dataframe2 = pd.read_csv('Csv-data/data-jumlah-kepala-keluarga-berdasarkan-jenis-kelamin-per-kelurahan-tahun-2020.csv')

data3 = dataframe2.loc[:,['kabupaten_kota','kecamatan','jumlah']]

In [ ]:
#data pre-processing
data3['jumlah'] = data3['jumlah'].astype(int)

data3_sum = data3.groupby('kecamatan')['jumlah'].transform('sum')
data3['jumlah_kk'] = data3_sum

In [31]:
#select only necessary and delete duplicate data
jumlah_kk = data3.loc[:,['kabupaten_kota','kecamatan','jumlah_kk']]
reduced_kk = jumlah_kk.drop_duplicates()

reduced_kk.to_csv('Csv-data/reduced-kk.csv',index=False)

In [ ]:
data_merge = pd.read_csv('Csv-data/merged.csv')
data_kk = pd.read_csv('Csv-data/reduced-kk-final.csv')
data_volume = pd.read_csv('Csv-data/reduced-volume-final.csv')

final_merged =

#### Data Visualization : Histogram

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

g

#### Data Visualization : Maps

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

#Prepare the maps
gdf_jakbar = gpd.read_file('city-regency-simplified/id-jk-jakbar.min.geojson')
gdf_jakpus = gpd.read_file('city-regency-simplified/id-jk-jakpus.min.geojson')
gdf_jaksel = gpd.read_file('city-regency-simplified/id-jk-jaksel.min.geojson')
gdf_jaktim = gpd.read_file('city-regency-simplified/id-jk-jaktim.min.geojson')
gdf_jakut = gpd.read_file('city-regency-simplified/id-jk-jakut.min.geojson')

#combine the maps
gdf_jakarta = gpd.GeoDataFrame(pd.concat([gdf_jakbar,gdf_jakpus,gdf_jaksel,gdf_jaktim,gdf_jakut], ignore_index=True), crs=gdf_jakbar.crs)

print(gdf_jakarta)

In [60]:
gdf1 = gpd.GeoDataFrame({'kecamatan':gdf_jakarta['name'],'geometry':gdf_jakarta['geometry']})
gdf2 = gdf1.sort_values(by='kecamatan')
print(gdf2)
datakk = pd.read_csv('Csv-data/reduced-kk-final.csv')

kk = datakk.loc[:,['kecamatan','jumlah_kk']]

kk['kecamatan'] = kk['kecamatan'].str.title()
kk['jumlah_kk'] = kk['jumlah_kk'].astype(int)

# print(gdf1)

kk1 = kk.sort_values(by='kecamatan')

#print(kk)

gdf_kk = gpd.GeoDataFrame({'kecamatan':gdf2['kecamatan'],'geometry':gdf2['geometry'],'jumlah_kk':kk1['jumlah_kk']})
#print(gdf_kk)



            kecamatan                                           geometry
35             Cakung  POLYGON ((106.90081 -6.21332, 106.90242 -6.206...
12      Cempaka Putih  POLYGON ((106.85532 -6.19216, 106.85979 -6.189...
0          Cengkareng  POLYGON ((106.70370 -6.17859, 106.70690 -6.176...
19           Cilandak  POLYGON ((106.76671 -6.29069, 106.77084 -6.288...
36          Cilincing  POLYGON ((106.90718 -6.10846, 106.90755 -6.103...
34           Cipayung  POLYGON ((106.88237 -6.29880, 106.89208 -6.298...
33            Ciracas  POLYGON ((106.86950 -6.30027, 106.87162 -6.296...
29        Duren Sawit  POLYGON ((106.89576 -6.21323, 106.90081 -6.213...
8              Gambir  POLYGON ((106.82850 -6.16262, 106.83015 -6.164...
1   Grogol Petamburan  POLYGON ((106.76779 -6.15784, 106.77522 -6.145...
21          Jagakarsa  POLYGON ((106.80909 -6.31462, 106.80805 -6.311...
28         Jatinegara  POLYGON ((106.85818 -6.21323, 106.86221 -6.211...
13         Johar Baru  POLYGON ((106.85220 -6.17451

In [ ]:
from matplotlib.colors import Normalize

gdf1 = gpd.GeoDataFrame({'kecamatan':gdf_jakarta['name']})

# Define the variable to use for coloring
variable = 'your_variable'

# Define the colormap and normalization
cmap = 'coolwarm'  # Example: Cool-Warm colormap
norm = Normalize(vmin=gdf1[variable].min(), vmax=gdf1[variable].max())

# Plot the GeoDataFrame with different colors based on the variable
gdf1.plot(column=variable, cmap=cmap, linewidth=0.8, edgecolor='black', norm=norm, legend=True)

# Customize the plot
plt.title('Map with Variable-based Colors')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show the colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
plt.colorbar(sm, label='Your Variable')

plt.show()
